In [1]:
import os
from pdf2image import convert_from_path
import pytesseract
from langchain.docstore.document import Document
from langchain.llms import WatsonxLLM
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ibm import WatsonxEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv

# Watsonx LLM
from langchain_ibm import WatsonxEmbeddings, WatsonxLLM
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import EmbeddingTypes
from ibm_granite_community.notebook_utils import get_env_var

from utils import buscar_cita_en_paginas 
load_dotenv()

True

In [3]:
# Definir el path del documento PDF
DOC_PATH = "Documento.pdf"
PAGES_CACHE = "pages.txt"

In [4]:
# Extraer el texto del PDF mediante OCR y guardarlo en caché para evitar reprocesos
if not os.path.exists(PAGES_CACHE):
    pages_images = convert_from_path(DOC_PATH, dpi=300)
    documents = []
    for i, image in enumerate(pages_images):
        # Ajusta 'lang' según el idioma (ej.: 'spa' para español)
        text = pytesseract.image_to_string(image, lang='spa')
        documents.append(Document(page_content=text, metadata={"page": i + 1}))
    with open(PAGES_CACHE, "w", encoding="utf-8") as f:
        f.write(str(documents))
    print(f"Documento procesado y guardado con {len(documents)} páginas.")
else:
    with open(PAGES_CACHE, "r", encoding="utf-8") as f:
        documents = eval(f.read())
    print(f"Documento cargado con {len(documents)} páginas desde {PAGES_CACHE}.")

# Unir todo el contenido en un solo string (opcional, para splitting global)
documento_completo = "\n".join([doc.page_content for doc in documents])

Documento cargado con 10 páginas desde pages.txt.


In [ ]:
import re
from transformers import pipeline, AutoTokenizer

# Inicializa el pipeline de NER y el tokenizer
ner_pipeline = pipeline(
    "ner", 
    model="mrm8488/bert-spanish-cased-finetuned-ner", 
    aggregation_strategy="simple",
)
tokenizer = AutoTokenizer.from_pretrained("mrm8488/bert-spanish-cased-finetuned-ner")

def limpiar_texto(texto):
    """
    Remueve caracteres extraños y normaliza espacios.
    """
    # Eliminar tokens tipo '##' y caracteres no alfabéticos (excepto espacios)
    texto = re.sub(r'##', '', texto)
    texto = re.sub(r'[^A-Za-zÁÉÍÓÚÑáéíóú\s]', '', texto)
    # Reemplaza múltiples espacios por uno solo
    texto = re.sub(r'\s+', ' ', texto)
    return texto.strip()

def extraer_firmantes(texto, max_length=512):
    """
    Divide el texto en fragmentos de a lo sumo max_length tokens y extrae las entidades de tipo PERSON.
    Aplica postprocesamiento para limpiar los resultados.
    Retorna una lista sin duplicados.
    """
    encoding = tokenizer(texto, add_special_tokens=True, truncation=False)
    token_ids = encoding["input_ids"]
    firmantes = []
    
    # Procesar en chunks basados en tokens
    for i in range(0, len(token_ids), max_length):
        chunk_ids = token_ids[i:i+max_length]
        chunk_text = tokenizer.decode(chunk_ids, skip_special_tokens=True)
        # Forzar UTF-8
        chunk_text = chunk_text.encode("utf-8").decode("utf-8")
        # Ejecutar NER en el fragmento
        entidades = ner_pipeline(chunk_text)
        for ent in entidades:
            if ent.get("entity_group") == "PER":
                nombre = limpiar_texto(ent["word"])
                # Filtrar nombres cortos o con caracteres sospechosos
                if len(nombre.split()) >= 2 and re.match(r'^[A-Z]', nombre):
                    firmantes.append(nombre)
                    
    return list(set(firmantes))

# Ejemplo de uso:
firmantes_detectados = extraer_firmantes(documento_completo)
print("Firmantes detectados:", firmantes_detectados)

Some weights of the model checkpoint at mrm8488/bert-spanish-cased-finetuned-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Firmantes detectados: ['IIS TOLEDO', 'Ia MOYA', 'Francisco Tapia Guerrero', 'S S']


In [ ]:
import nest_asyncio
nest_asyncio.apply()

from dotenv import load_dotenv
load_dotenv()

# bring in deps
from llama_cloud_services import LlamaParse
from llama_index.core import SimpleDirectoryReader

# set up parser
parser = LlamaParse(
    result_type="markdown" 
)

# use SimpleDirectoryReader to parse our file
file_extractor = {".pdf": parser}
documents = SimpleDirectoryReader(input_files=['Documento.pdf'], file_extractor=file_extractor).load_data()

print(documents)

Started parsing the file under job_id 6fd19ef9-ec57-4921-ad46-eea4b82f3d52


In [ ]:
from pyhanko.pdf_utils.reader import PdfFileReader
from pyhanko.sign.validation import validate_pdf_signature
from pyhanko_certvalidator import ValidationContext

def extraer_firmantes_digitales(pdf_path):
    firmantes = []
    with open(pdf_path, 'rb') as f:
        reader = PdfFileReader(f)
        # Crea un contexto de validación (puedes ajustarlo según tus necesidades)
        vc = ValidationContext()
        # Itera sobre todas las firmas embebidas en el PDF
        for sig in reader.embedded_signatures:
            try:
                # Valida la firma
                status = validate_pdf_signature(sig, vc)
                # Si se pudo validar y existe el certificado del firmante, extrae el nombre
                if status.signing_cert:
                    firmante = status.signing_cert.subject.human_friendly
                    firmantes.append(firmante)
            except Exception as e:
                print("Error al validar firma:", e)
    return firmantes

if __name__ == '__main__':
    pdf = "Documento.pdf"
    print("Firmantes digitales detectados:")
    for firmante in extraer_firmantes_digitales(pdf):
        print(firmante)


Firmantes digitales detectados:
